In [3]:
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [17]:
# load data set
x_import = pandas.read_csv('/Users/kimardenmiller/dropbox/tensorflow/data/x2015_noFinance.csv', header=None)

# split into input (X) and output (Y) variables
x_data_values = x_import.values
x_data_feature_values = x_data_values[:, [0, 6, 8, 9, 13, 17, 18, 20]]
x_data_features = x_data_feature_values[1:, :]   # Remove Labels
x_data_features[x_data_features == ''] = 0.0    # Remove Blanks
selected_feature_labels = x_data_feature_values[0, 1:]
print('Selected Feature Names: \n', selected_feature_labels)
print('First few Stocks with Features, no Labels: ', '\n', x_data_features[0:2, :], ' ...')
print(np.size(x_data_features[:, 0]), 'Stocks by', np.size(x_data_features[0, :]), 'Features (with Tickers')

y_import = pandas.read_csv('/Users/kimardenmiller/dropbox/tensorflow/data/y201501_noFinancials.csv', header=None)
y_data_values = y_import[1:].values

x_tickers = x_data_features[:, 0]
print('x tickers: ', x_tickers)
y_tickers = y_data_values[:, 0]
print('Total Y Tickers: ', np.size(y_tickers))
print('First few Y tickers: \n', y_tickers[0:5])

# Format Y to y = 1 (positive) and y = 0 (negative) examples 
true_false_mask = np.in1d(x_tickers, y_tickers)
y_mask = np.where(true_false_mask, 1, 0)
print('Total Positive Y Ticker Example Count: ', np.size(np.nonzero(y_mask)), )
print('Total Positive Y Ticker Example Count on x_tickers: ', np.size(x_tickers[np.nonzero(y_mask)]))
print('Total Y Ticker Mask Count: ', np.size(y_mask))
print('Y after Encoding: ', y_mask[0:100])

# Place dataset into input (X) and output (Y) variables
x_strings = x_data_features[:, 1:]  # take off tickers, as they can't be tensor'd
raw_X = x_strings.astype(np.float)  # convert strings to float
print('Training Examples: ', np.size(raw_X[:, 0]), ' x ', np.size(raw_X[0, :]), ' Features: ')

X = raw_X       # X value assigned
Y = y_mask      # Y uses the 0, 1 to show negative and positive examples
np.set_printoptions(precision=3, suppress=True)
print('First few X Training Examples with', np.size(raw_X[0, :]), '\nSelected Features: ', raw_X[0:2, :], ' ...')

Selected Feature Names: 
 ['total yield' 'roic v2' 'book price' 'fcf yield' 'total return'
 'spitz roic' 'momentum']
First few Stocks with Features, no Labels:  
 [['DD' '0.03' '0.19' '0.16' '0.03' '0.09' '0.18' '0.93']
 ['DOW' '0.04' '0.13' '0.42' '0.03' '0.1' '0.13' '0.94']]  ...
1436 Stocks by 8 Features (with Tickers
x tickers:  ['DD' 'DOW' 'MON' ..., 'CTWS' 'UTL' 'SPKE']
Total Y Tickers:  237
First few Y tickers: 
 ['AAPL' 'ABAX' 'ABC' 'AFB' 'AGU']
Total Positive Y Ticker Example Count:  119
Total Positive Y Ticker Example Count on x_tickers:  119
Total Y Ticker Mask Count:  1436
Y after Encoding:  [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Training Examples:  1436  x  7  Features: 
First few X Training Examples with 7 
Selected Features:  [[ 0.03  0.19  0.16  0.03  0.09  0.18  0.93]
 [ 0.04  0.13  0.42  0.03  0.1   0.13  

In [3]:
# encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(Y)
# encoded_Y = encoder.transform(Y)
print('Pre Encoding: ', Y[90:100], 'After Encoding: ', encoded_Y[90:100])

Pre Encoding:  ['R' 'R' 'R' 'R' 'R' 'R' 'R' 'M' 'M' 'M'] After Encoding:  [1 1 1 1 1 1 1 0 0 0]


In [18]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(7, input_dim=7, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# evaluate model with standardized data set
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# evaluate baseline model with standardized data set
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
186/186 [==============================] - 0s - loss: 0.6487 - acc: 0.6559     
Epoch 2/100
186/186 [==============================] - 0s - loss: 0.5502 - acc: 0.7957     
Epoch 3/100
186/186 [==============================] - 0s - loss: 0.4604 - acc: 0.7903     
Epoch 4/100
186/186 [==============================] - 0s - loss: 0.3847 - acc: 0.8495     
Epoch 5/100
186/186 [==============================] - 0s - loss: 0.3234 - acc: 0.8871     
Epoch 6/100
186/186 [==============================] - 0s - loss: 0.2766 - acc: 0.9140     
Epoch 7/100
186/186 [==============================] - 0s - loss: 0.2382 - acc: 0.9247     
Epoch 8/100
186/186 [==============================] - 0s - loss: 0.2049 - acc: 0.9570     
Epoch 9/100
186/186 [==============================] - 0s - loss: 0.1784 - acc: 0.9624     
Epoch 10/100
186/186 [==============================] - 0s - loss: 0.1525 - acc: 0.9892     
Epoch 11/100
186/186 [==============================] - 0s - loss: 0.1338 - acc

In [6]:
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 77.82% (7.00%)


In [7]:
# smaller model
def create_smaller():
	# create model
	model = Sequential()
	model.add(Dense(30, input_dim=60, init='normal', activation='relu'))
	model.add(Dense(1, init='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 82.68% (4.38%)


In [20]:
# larger model
def create_larger():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
	model.add(Dense(30, init='normal', activation='relu'))
	model.add(Dense(1, init='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 84.57% (3.72%)


In [22]:
# more epochs
def create_larger():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
	model.add(Dense(30, init='normal', activation='relu'))
	model.add(Dense(1, init='normal', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, nb_epoch=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger & Deeper: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger & Deeper: 83.11% (5.45%)
